# Анализ базы данных сервиса для чтения книг по подписке.

По причине покупки крупного сервиса для чтения книг по подписке необходимо проанализировать его базу данных. 

База данных содержит информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

Для решения поставленной задачи будут изучаться 5 таблиц со следующей структурой:

1. **Таблица books**

    Содержит данные о книгах:
    - book_id — идентификатор книги;
    - author_id — идентификатор автора;
    - title — название книги;
    - num_pages — количество страниц;
    - publication_date — дата публикации книги;
    - publisher_id — идентификатор издателя.
    
    
2. **Таблица authors**

    Содержит данные об авторах:
    - author_id — идентификатор автора;
    - author — имя автора.
    
    
3. **Таблица publishers**

    Содержит данные об издательствах:
    - publisher_id — идентификатор издательства;
    - publisher — название издательства.
    
    
4. **Таблица ratings**

    Содержит данные о пользовательских оценках книг:
    - rating_id — идентификатор оценки;
    - book_id — идентификатор книги;
    - username — имя пользователя, оставившего оценку;
    - rating — оценка книги.
    
    
5. **Таблица reviews**

    Содержит данные о пользовательских обзорах:
    - review_id — идентификатор обзора;
    - book_id — идентификатор книги;
    - username — имя автора обзора;
    - text — текст обзора.

## Подключаемся к базе данных.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': '...', # пароль
'host': '...',
'port': ..., # порт подключения
'db': '...'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Просмотр таблиц

На данном этапе выведем по 5 первых строк для каждой из таблиц, описание структуры данных содержится выше в введении.

In [3]:
query = '''SELECT * 
           FROM books 
           LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


База данных настроена, первый проверочный запрос обработан - выведено 5 первых строк из таблицы `books`. Выведем 5 первых строк из других табллиц.

In [4]:
query = '''SELECT * 
           FROM authors 
           LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
query = '''SELECT * 
           FROM publishers 
           LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
query = '''SELECT * 
           FROM ratings 
           LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
query = '''SELECT * 
           FROM reviews 
           LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Анализ данных

**Посчитайте, сколько книг вышло после 1 января 2000 года.**

In [8]:
query = '''SELECT COUNT(book_id) 
           FROM books 
           WHERE publication_date >= '2000-01-01' '''
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,821


In [9]:
query = '''SELECT COUNT(book_id) 
           FROM books '''
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,1000


Итак, после 1 января 2000 года вышло 821 книги - больше 80% от всех книг, имеющихся в купленном сервисе для чтения книг (всего их 1000). 

**Для каждой книги посчитайте количество обзоров и среднюю оценку.**

In [10]:
query = '''SELECT b.title,
                  AVG(rating) AS rating,
                  COUNT(DISTINCT (review_id)) AS review
           FROM books AS b
           FULL OUTER JOIN ratings AS ra ON b.book_id=ra.book_id
           FULL OUTER JOIN reviews AS re ON b.book_id=re.book_id
           GROUP BY b.book_id
           ORDER BY review'''
pd.io.sql.read_sql(sql=text(query), con = con)

,title,rating,review
0,Anne Rice's The Vampire Lestat: A Graphic Novel,3.666667,0
1,The Cat in the Hat and Other Dr. Seuss Favorites,5.000000,0
2,Essential Tales and Poems,4.000000,0
3,The Natural Way to Draw,3.000000,0
4,Disney's Beauty and the Beast (A Little Golden...,4.000000,0
...,...,...,...
995,The Giver (The Giver #1),3.750000,6
996,The Road,3.772727,6
997,The Curious Incident of the Dog in the Night-Time,4.081081,6
998,The Da Vinci Code (Robert Langdon #2),3.830508,6


Итак, для того, чтобы ответить на поставленный вопрос, мы соединили 3 таблицы с необходимыми для нас данными и вывели следующие столбцы, сгруппировав все по book_id - название книги, средний рейтинг и количество отзывов. Самой популярной книгой оказалась первая часть Сумерек со средним рейтингом в 3.66.

**Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры.**

In [11]:
query = '''SELECT publisher,
                  COUNT(book_id) AS amount
           FROM books AS b 
           LEFT JOIN publishers AS p ON b.publisher_id=p.publisher_id 
           WHERE num_pages > 50
           GROUP BY publisher
           ORDER BY amount DESC
           LIMIT 1'''
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,amount
0,Penguin Books,42


Итак, для того, чтобы найти самое крупное издательство, необходимо было объединить 2 таблицы, отсечь издательства, выпускающие брошюры, посчитать количество изданных книг и отсортировать по уменьшению. По полученным данным самым крупным издательством в наших данных является Pinguin Books - ими выпущено на момент анализа 42 книги.

**Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.**

In [12]:
query = '''WITH
           r AS 
           (SELECT book_id,
                  COUNT(rating_id) AS amount,
                  AVG(rating) AS rating
           FROM ratings 
           GROUP BY book_id)
           
           SELECT author,
                  AVG(rating) AS avg_rating
           FROM r
           LEFT JOIN books AS b ON r.book_id=b.book_id
           LEFT JOIN authors AS a ON b.author_id=a.author_id
           WHERE amount > 50
           GROUP BY author
           ORDER BY avg_rating DESC
           LIMIT 1
           '''
pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


Для того, чтобы определеить автора с самой высокой средней оценкой книг мы сделали следующее - сначала посчитали средние оценки каждой книги и сколько оценок она вообще имеет, потом указанные данные соединили с таблицами по книгам и по авторам, повторно нашли для всех авторов среднюю оценку всех книг и отфильтровали автора с лучшим показателем - им, как весьма неудивительно, оказалась Джоан Роулинг со средним рейтингом книг в 4.2.

**Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.**

In [13]:
query = '''WITH
           users AS
           (WITH
           ra AS
           (SELECT COUNT(rating_id) AS amount,
                  username
           FROM ratings 
           GROUP BY username)
           
           SELECT ra.username,
                  COUNT(re.review_id) AS amount
           FROM ra
           LEFT JOIN reviews AS re ON ra.username=re.username
           WHERE amount > 48
           GROUP BY ra.username)
           
           SELECT AVG(amount) AS avg_amount_reviews
           FROM users'''
pd.io.sql.read_sql(sql=text(query), con = con)

,avg_amount_reviews
0,24.0


Для того, чтобы выяснить, какое среднее количество обзоров от пользователей, нужно сделать следующее: сначала найти, какое количество оценок поставил каждый юзер и выбрать тех, кто оставил больше 48. Далее считаем у этих пользователей количество обзоров. Завершающим этапом становится поиск среднего. Таким образом, пользователи, которые поставили больше 48 оценок, в среднем написали 24 обзора на книги.